# HYPER PARAMETERS FINE TUNING USING OPTUNA

In [ ]:
!pip install optuna

     |████████████████████████████████| 308 kB 4.3 MB/s 
     |████████████████████████████████| 81 kB 8.5 MB/s 
     |████████████████████████████████| 210 kB 47.4 MB/s 
     |████████████████████████████████| 78 kB 5.6 MB/s 
     |████████████████████████████████| 49 kB 5.8 MB/s 
     |████████████████████████████████| 146 kB 47.8 MB/s 
     |████████████████████████████████| 113 kB 44.5 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=5faddd150096bcd732d2f1568496dc96b64b2fa9179839d313ba12ce684906d1
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [ ]:
# LIBRARIES

import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
import xgboost as xgb
import optuna
from sklearn.metrics import mean_squared_error


from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/MyDrive/Colab_Notebooks/Proyecto_final"

Mounted at /content/drive
/content/drive/MyDrive/Colab_Notebooks/Proyecto_final


In [ ]:
# !pip install xgboost

In [ ]:
# DEFINE METRICS
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score

In [ ]:
def get_metrics(y_true, y_pred, model_name):

    RMSE = mean_squared_error(y_true, y_pred, squared=False)
    MAE = mean_absolute_error(y_true, y_pred)
    EVS = explained_variance_score(y_true, y_pred)
    R2 = r2_score(y_true, y_pred)
    print('El MSE obtenido con ' + model_name + ' es: ' + str(RMSE))
    print('El MAE obtenido con ' + model_name + ' es: ' + str(MAE))
    print('El EVS obtenido con ' + model_name + ' es: ' + str(EVS))
    print('El R2 obtenido con ' + model_name + ' es: ' + str(R2))
    
    return [model_name, RMSE, MAE, EVS, R2]

In [ ]:
!pip3 install pickle5
import pickle5 as pickle

     |████████████████████████████████| 256 kB 4.4 MB/s 


In [ ]:
# IMPORT DATA

# train_set = pd.read_pickle('./TRAIN_TO_MODEL_20220212.pkl')
# test_set = pd.read_pickle('./TEST_TO_MODEL_20220212.pkl')

with open('./TRAIN_TO_MODEL_20220212.pkl', "rb") as fh:
  train_set = pickle.load(fh)
with open('./TEST_TO_MODEL_20220212.pkl', "rb") as fh:
  test_set = pickle.load(fh)

# Split train and test datasets

X = train_set.drop(columns = ['MG_NIR_Salida_Ext_FUT30', 'MG_NIR_Salida_Ext_FUT15', 'MG_NIR_Salida_Ext_FUT3','Humedad_Semilla_LAG_FUT3', 'T_Lam_9_LAG_FUT3',
       'Humedad_Semilla_LAG_FUT15', 'T_Lam_9_LAG_FUT15'])
y = train_set['MG_NIR_Salida_Ext_FUT30']

X_test = test_set.drop(columns = ['MG_NIR_Salida_Ext_FUT30', 'MG_NIR_Salida_Ext_FUT15', 'MG_NIR_Salida_Ext_FUT3','Humedad_Semilla_LAG_FUT3', 'T_Lam_9_LAG_FUT3',
       'Humedad_Semilla_LAG_FUT15', 'T_Lam_9_LAG_FUT15'])
y_test = test_set['MG_NIR_Salida_Ext_FUT30']


# Split train and validation datasets
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.20, random_state = 42)

In [ ]:
X_train.head()

,TT-35-GI,Humedad_Semilla,T_Mat_entrada,T_Solvente,Q_Solvente,T_Lam_9,Ritmo_Molienda,Q_Miscela,Vel_Extractor,Conc_Miscela,MG_NIR_Salida_Ext,Solvente_NIR_Salida_Ext,LT-17,Conc_P3-11,Humedad_Semilla_LAG,T_Lam_9_LAG,MG_NIR_Salida_Ext_LAG3,MG_NIR_Salida_Ext_LAG15,MG_NIR_Salida_Ext_LAG30
2887,0.509415,0.638268,0.664635,0.373273,0.750889,0.503741,0.100160,0.830802,0.797707,0.647806,0.494274,0.383620,0.264080,0.502716,0.501086,0.682260,0.490006,0.494618,0.458558
2715,0.231501,0.278769,0.205871,0.265344,0.234343,0.483056,0.728122,0.284815,0.334171,0.438703,0.476502,0.521832,0.667808,0.561199,0.441831,0.457732,0.497817,0.527552,0.550907
818,0.408723,0.298984,0.374659,0.717798,0.635906,0.434130,0.330496,0.292308,0.576712,0.589926,0.408342,0.563025,0.587833,0.486247,0.410062,0.425975,0.385737,0.381034,0.424934
3559,0.485359,0.856388,0.431049,0.620287,0.599326,0.587839,0.666808,0.709947,0.652226,0.240418,0.624979,0.475601,0.474736,0.458049,0.637721,0.672518,0.631905,0.596820,0.539860
2578,0.309565,0.575917,0.142758,0.652683,0.790588,0.498621,0.607786,0.735618,0.702019,0.421516,0.433869,0.559271,0.533003,0.669441,0.559145,0.447872,0.420163,0.427312,0.437913


In [ ]:
def objective(trial, X_train = X_train, X_val = X_val, y_train = y_train, y_val = y_val, es_n_rounds = 10, target = y_train):

    param = {
        'tree_method':'gpu_hist',  # this parameter means using the GPU when training our model to speedup the training process

        'booster' : 'dart',       # DART HPs
        'sample_type' : trial.suggest_categorical('sample_type', ['uniform' , 'weighted']),
        'normalize_type' : trial.suggest_categorical('normalize_type', ['tree' , 'forest']),
        'rate_drop' : trial.suggest_loguniform('rate_drop', 1e-3, 1.0),
        'one_drop': trial.suggest_categorical('one_drop', [0, 1]),

        'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.008,0.009,0.01,0.012,0.014,0.016,0.018, 0.02]),
        'n_estimators': trial.suggest_categorical('n_estimators', [10, 100, 500, 1000, 2000]),
        'max_depth': trial.suggest_categorical('max_depth', [5,7,9,11,13,15,17,20]),
        'random_state': trial.suggest_categorical('random_state', [24, 48,2020]),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
    }
    model = xgb.XGBRegressor(**param)  
    
    model.fit(X_train, y_train, eval_set = [(X_val, y_val)], early_stopping_rounds = es_n_rounds, verbose = False)
    
    preds = model.predict(X_val)
    
    rmse = mean_squared_error(y_val, preds,squared=False)
    
    return rmse

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2022-04-17 21:32:44,981] A new study created in memory with name: no-name-f9e59ee2-cffa-4486-990a-c9f67ef6067e


[21:32:45] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2022-04-17 21:32:45,840] Trial 0 finished with value: 0.09667570419858533 and parameters: {'sample_type': 'uniform', 'normalize_type': 'forest', 'rate_drop': 0.001826445796852355, 'one_drop': 0, 'lambda': 0.09540139451025204, 'alpha': 0.05796873858768422, 'colsample_bytree': 1.0, 'subsample': 1.0, 'learning_rate': 0.008, 'n_estimators': 10, 'max_depth': 15, 'random_state': 24, 'min_child_weight': 203}. Best is trial 0 with value: 0.09667570419858533.


[21:32:45] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2022-04-17 21:32:46,291] Trial 1 finished with value: 0.07825386122735052 and parameters: {'sample_type': 'uniform', 'normalize_type': 'forest', 'rate_drop': 0.3371307464668547, 'one_drop': 1, 'lambda': 0.02730882023994712, 'alpha': 0.019129512241816398, 'colsample_bytree': 0.9, 'subsample': 0.4, 'learning_rate': 0.02, 'n_estimators': 2000, 'max_depth': 5, 'random_state': 2020, 'min_child_weight': 168}. Best is trial 1 with value: 0.07825386122735052.


...

[22:52:51] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2022-04-17 22:54:44,395] Trial 48 finished with value: 0.03534755157855055 and parameters: {'sample_type': 'weighted', 'normalize_type': 'forest', 'rate_drop': 0.006595655434053008, 'one_drop': 0, 'lambda': 0.18763439590418682, 'alpha': 0.0041931768421324105, 'colsample_bytree': 0.6, 'subsample': 0.5, 'learning_rate': 0.02, 'n_estimators': 1000, 'max_depth': 15, 'random_state': 24, 'min_child_weight': 28}. Best is trial 21 with value: 0.0326532375561884.


[22:54:44] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2022-04-17 22:54:44,749] Trial 49 finished with value: 0.09674880764439905 and parameters: {'sample_type': 'weighted', 'normalize_type': 'forest', 'rate_drop': 0.009857039903555248, 'one_drop': 0, 'lambda': 0.3698605421258651, 'alpha': 0.0011041610868283973, 'colsample_bytree': 0.6, 'subsample': 0.8, 'learning_rate': 0.008, 'n_estimators': 10, 'max_depth': 7, 'random_state': 24, 'min_child_weight': 63}. Best is trial 21 with value: 0.0326532375561884.


Number of finished trials: 50
Best trial: {'sample_type': 'uniform', 'normalize_type': 'forest', 'rate_drop': 0.0033681936632674442, 'one_drop': 0, 'lambda': 0.0013776246362593974, 'alpha': 0.009577373870014602, 'colsample_bytree': 0.3, 'subsample': 0.7, 'learning_rate': 0.009, 'n_estimators': 1000, 'max_depth': 13, 'random_state': 24, 'min_child_weight': 6}


In [ ]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_alpha,params_colsample_bytree,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_n_estimators,params_normalize_type,params_one_drop,params_random_state,params_rate_drop,params_sample_type,params_subsample,state
0,0,0.096676,2022-04-17 21:32:45.001988,2022-04-17 21:32:45.839889,0 days 00:00:00.837901,0.057969,1.0,0.095401,0.008,15,203,10,forest,0,24,0.001826,uniform,1.0,COMPLETE
1,1,0.078254,2022-04-17 21:32:45.843628,2022-04-17 21:32:46.290597,0 days 00:00:00.446969,0.019130,0.9,0.027309,0.020,5,168,2000,forest,1,2020,0.337131,uniform,0.4,COMPLETE
2,2,0.093200,2022-04-17 21:32:46.297129,2022-04-17 21:32:46.527056,0 days 00:00:00.229927,0.008370,0.7,9.646129,0.016,15,97,10,forest,1,48,0.088251,weighted,0.5,COMPLETE
3,3,0.060300,2022-04-17 21:32:46.530113,2022-04-17 21:32:47.685848,0 days 00:00:01.155735,0.002644,0.7,0.001807,0.020,7,177,2000,forest,1,24,0.196223,weighted,0.6,COMPLETE
4,4,0.040715,2022-04-17 21:32:47.689314,2022-04-17 21:33:46.129214,0 days 00:00:58.439900,0.034434,0.4,0.002300,0.010,7,20,1000,tree,0,24,0.005430,uniform,0.7,COMPLETE
5,5,0.048326,2022-04-17 21:33:46.132984,2022-04-17 21:34:02.598122,0 days 00:00:16.465138,0.001556,0.6,0.152435,0.008,5,55,2000,forest,1,48,0.027726,uniform,0.8,COMPLETE
6,6,0.061352,2022-04-17 21:34:02.600360,2022-04-17 21:34:09.665279,0 days 00:00:07.064919,0.215382,0.3,6.928886,0.012,13,215,500,tree,1,48,0.025100,uniform,1.0,COMPLETE
7,7,0.059509,2022-04-17 21:34:09.667558,2022-04-17 21:34:11.299815,0 days 00:00:01.632257,2.603356,0.4,3.827893,0.020,11,136,2000,forest,0,24,0.222229,weighted,0.6,COMPLETE
8,8,0.092718,2022-04-17 21:34:11.302729,2022-04-17 21:34:11.556668,0 days 00:00:00.253939,0.010441,0.6,2.809562,0.016,17,191,10,tree,0,24,0.001683,weighted,0.7,COMPLETE
9,9,0.099363,2022-04-17 21:34:11.561584,2022-04-17 21:34:11.905209,0 days 00:00:00.343625,1.961672,0.4,0.082363,0.008,20,50,10,tree,1,48,0.408054,weighted,0.5,COMPLETE


In [ ]:
#plot_optimization_histor: shows the scores from all trials as well as the best score so far at each point.
optuna.visualization.plot_optimization_history(study)

In [ ]:
#plot_parallel_coordinate: interactively visualizes the hyperparameters and scores
optuna.visualization.plot_parallel_coordinate(study)

In [ ]:
optuna.visualization.plot_contour(study, params = ['n_estimators', 'rate_drop', 'max_depth'])

In [ ]:
# plot_slice: shows the evolution of the search. You can see where in the hyperparameter space your search 
# went and which parts of the space were explored more.
optuna.visualization.plot_slice(study)

In [ ]:
#Visualize parameter importances.
optuna.visualization.plot_param_importances(study)

In [ ]:
#Visualize empirical distribution function
optuna.visualization.plot_edf(study)

## TEST SET

In [ ]:
print('Best trial:', study.best_trial.params)

Best trial: {'sample_type': 'uniform', 'normalize_type': 'forest', 'rate_drop': 0.0033681936632674442, 'one_drop': 0, 'lambda': 0.0013776246362593974, 'alpha': 0.009577373870014602, 'colsample_bytree': 0.3, 'subsample': 0.7, 'learning_rate': 0.009, 'n_estimators': 1000, 'max_depth': 13, 'random_state': 24, 'min_child_weight': 6}


In [ ]:
# Final model

XGB_model = xgb.XGBRegressor( **study.best_trial.params)
XGB_model.fit(X_train, y_train)

[23:46:21] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(alpha=0.009577373870014602, colsample_bytree=0.3,
             lambda=0.0013776246362593974, learning_rate=0.009, max_depth=13,
             min_child_weight=6, n_estimators=1000, normalize_type='forest',
             one_drop=0, random_state=24, rate_drop=0.0033681936632674442,
             sample_type='uniform', subsample=0.7)

In [ ]:
y_pred_val = XGB_model.predict(X_val)
get_metrics(y_val, y_pred_val, 'val_scores')

El MSE obtenido con val_scores es: 0.032339657031884274
El MAE obtenido con val_scores es: 0.021191131551880987
El EVS obtenido con val_scores es: 0.8996531653028376
El R2 obtenido con val_scores es: 0.89961316875221


['val_scores',
 0.032339657031884274,
 0.021191131551880987,
 0.8996531653028376,
 0.89961316875221]

In [ ]:
y_pred_test = XGB_model.predict(X_test)
get_metrics(y_test, y_pred_test, 'val_scores')

El MSE obtenido con val_scores es: 0.044016669622910766
El MAE obtenido con val_scores es: 0.035231111912300064
El EVS obtenido con val_scores es: 0.4265072914699941
El R2 obtenido con val_scores es: 0.4188142857727898


['val_scores',
 0.044016669622910766,
 0.035231111912300064,
 0.4265072914699941,
 0.4188142857727898]

In [ ]:
column_names = ['y_ACT', 'y_FUT_30', 'XGB DART FT']
test_pred_DF = pd.DataFrame(columns = column_names)

test_pred_DF['y_ACT'] = X_test['MG_NIR_Salida_Ext']
test_pred_DF['y_FUT_30'] = y_test
test_pred_DF['XGB DART FT'] = XGB_model.predict(X_test)

In [ ]:
pd.options.plotting.backend = "plotly"
test_pred_DF.plot()